In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [14]:
train = pd.read_csv(r"dataset/train.csv")
train = np.array(train)

In [25]:
X = train[1:, 1:-1]
y = train[1:, -1]

TypeError: object of type 'int' has no len()